In [1]:
df = spark.sql("SELECT * FROM News_lake_db.news_updated_data ")
display(df.limit(7))

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 96c941b9-7a8a-482b-b821-e39bcfd41159)

In [2]:
#import required packages
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 4, Finished, Available, Finished)

In [3]:
#initialize the model and configure the input and output columns
'''model = (AnalyzeText()
        .setTextCol("Choose the Text column")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error")) 
'''

StatementMeta(, , , Waiting, , Waiting)

In [3]:
#initialize the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 5, Finished, Available, Finished)

In [4]:
#Apply the model to our dataframe
result = model.transform(df)

display(result.limit(10))

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1b763d36-af69-44da-9e9d-2bda574cdbff)

In [5]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))
display(sentiment_df.limit(7))

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d6a02fb6-90b2-4005-8b99-75184d908496)

In [6]:
#Drop error and response columns

sentiment_df = sentiment_df.drop("error","response")
display(sentiment_df.limit(7))

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 37b7baaf-82c3-4e97-a5bd-1da2e65bb45b)

In [7]:
#Adopting TYPE 1 Match in Data Warehousing

'''a Type 1 match refers to a scenario where the new data iss appended to an existing data without duplicate. 
This is typically used when there is no need to keep a history of the data.
'''
from pyspark.sql.utils import AnalysisException

#Exception Handling
try:

    table_name = "News_lake_db.sentiment_analysis"
    sentiment_df.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print ("Table Already Exist")

    sentiment_df.createOrReplaceTempView("vw_sentiment")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment source_view

                    ON source_view.link = target_table.link

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.category <> target_table.category OR
                    source_view.country <> target_table.country OR
                    source_view.description <> target_table.description OR
                    source_view.keywords <> target_table.keywords OR
                    source_view.language <> target_table.language OR
                    source_view.link <> target_table.link OR
                    source_view.pubDate <> target_table.pubDate OR
                    source_view.Date <> target_table.Date OR
                    source_view.Time <> target_table.Time
                    
                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT * 

                """)

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 9, Finished, Available, Finished)

In [8]:
%%sql

SELECT COUNT(*) FROM News_lake_db.sentiment_analysis

StatementMeta(, 5b44fecc-8aec-457d-a620-e8373874f9e9, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>